In [1]:
# Note: The openai-python library support for Azure OpenAI is in preview.
import configparser
import os

import openai
import requests
from PIL import Image

config = configparser.ConfigParser()
config.read("config.ini")


engine = ""


def config_model(section="inm.gpt4"):
    openai.api_key = config[section]["api_key"]
    openai.api_type = config[section]["api_type"]
    openai.api_base = config[section]["api_base"]
    openai.api_version = config[section]["api_version"]
    global engine
    engine = config[section]["engine"]


config_model()  # default gpt4

In [10]:
def search_hotels():
    return True


from datetime import datetime

import pytz


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."


functions = [
    {
        "name": "search_hotels",
        "description": "Retrieves hotels from the search index based on the parameters provided",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string", "description": "The name of the hotel"},
                "features": {
                    "type": "string",
                    "description": "A comma separated list of features (i.e. beachfront, free wifi, etc.)",
                },
            },
            "required": ["name", "features"],
        },
    },
    {
        "name": "get_current_time",
        "description": "Get the current time in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                }
            },
            "required": ["location"],
        },
    },
]

available_functions = {
    "name": search_hotels,
    "get_current_time": get_current_time,
}

In [11]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [12]:
import json


def run_conversation(messages, functions, available_functions, engine):
    response = openai.ChatCompletion.create(
        engine=engine,
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()

        function_name = response_message["function_call"]["name"]

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages, engine=engine
        )  # get a new response from GPT where it can see the function response

        return second_response

In [13]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, functions, available_functions, engine)
print(assistant_response["choices"][0]["message"])

Recommended Function call:
{
  "name": "get_current_time",
  "arguments": "{\n  \"location\": \"America/New_York\"\n}"
}

Output of function call:
05:22:06 AM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'name': 'get_current_time', 'content': '{\n  "location": "America/New_York"\n}'}
{'role': 'function', 'name': 'get_current_time', 'content': '05:22:06 AM'}

{
  "role": "assistant",
  "content": "It is currently 05:22 AM in New York."
}
